In [1]:
# Built-in library
import re
import json
import logging
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
import polars as pl
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

### Create A DataFrame

In [2]:
from datetime import datetime


df: pl.DataFrame = pl.DataFrame(
    {
        "integer": [1, 2, 3, 4, 5],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
            datetime(2022, 1, 4),
            datetime(2022, 1, 5),
        ],
        "float": [4.0, 5.0, 6.0, 7.0, 8.0],
    }
)

print(df)

shape: (5, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ datetime[μs]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
│ 4       ┆ 2022-01-04 00:00:00 ┆ 7.0   │
│ 5       ┆ 2022-01-05 00:00:00 ┆ 8.0   │
└─────────┴─────────────────────┴───────┘

#### Viewing Data

```text
- df.head()
- df.tail()
```

In [3]:
df.head(2)

integer,date,float
i64,datetime[μs],f64
1,2022-01-01 00:00:00,4.0
2,2022-01-02 00:00:00,5.0


In [4]:
df.tail(2)

integer,date,float
i64,datetime[μs],f64
4,2022-01-04 00:00:00,7.0
5,2022-01-05 00:00:00,8.0


#### Descriptive Stats

In [5]:
df.describe()

describe,integer,date,float
str,f64,str,f64
"""count""",5.0,"""5""",5.0
"""null_count""",0.0,"""0""",0.0
"""mean""",3.0,null,6.0
"""std""",1.581139,null,1.581139
"""min""",1.0,"""2022-01-01 00:…",4.0
"""25%""",2.0,null,5.0
"""50%""",3.0,null,6.0
"""75%""",4.0,null,7.0
"""max""",5.0,"""2022-01-05 00:…",8.0


In [6]:
df: pl.DataFrame = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.938371 ┆ A      │
│ 2    ┆ ham   ┆ 0.11866  ┆ A      │
│ 3    ┆ spam  ┆ 0.575215 ┆ B      │
│ null ┆ egg   ┆ 0.859469 ┆ C      │
│ 5    ┆ null  ┆ 0.020818 ┆ B      │
└──────┴───────┴──────────┴────────┘

In [7]:
# Select
out: pl.DataFrame = df.select(
    pl.sum("nrs"),  # sum this column
    pl.col("names").sort(),  # select and sort this column
    # select the first row and rename this column
    pl.col("names").first().alias("first name"),
    # calculate the mean, multiply by 10 and rename
    (pl.mean("nrs") * 10).alias("10xnrs"),
)
print(out)

shape: (5, 4)
┌─────┬───────┬────────────┬────────┐
│ nrs ┆ names ┆ first name ┆ 10xnrs │
│ --- ┆ ---   ┆ ---        ┆ ---    │
│ i64 ┆ str   ┆ str        ┆ f64    │
╞═════╪═══════╪════════════╪════════╡
│ 11  ┆ null  ┆ foo        ┆ 27.5   │
│ 11  ┆ egg   ┆ foo        ┆ 27.5   │
│ 11  ┆ foo   ┆ foo        ┆ 27.5   │
│ 11  ┆ ham   ┆ foo        ┆ 27.5   │
│ 11  ┆ spam  ┆ foo        ┆ 27.5   │
└─────┴───────┴────────────┴────────┘

In [9]:
df

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.938371,"""A"""
2,"""ham""",0.11866,"""A"""
3,"""spam""",0.575215,"""B"""
null,"""egg""",0.859469,"""C"""
5,null,0.020818,"""B"""


In [8]:
# Add new column(s)
df_1: pl.DataFrame = df.with_columns(
    pl.sum("nrs").alias("nrs_sum"),
    pl.col("random").count().alias("count"),
)
print(df_1)

shape: (5, 6)
┌──────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs  ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ ---  ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64  ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞══════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 1    ┆ foo   ┆ 0.938371 ┆ A      ┆ 11      ┆ 5     │
│ 2    ┆ ham   ┆ 0.11866  ┆ A      ┆ 11      ┆ 5     │
│ 3    ┆ spam  ┆ 0.575215 ┆ B      ┆ 11      ┆ 5     │
│ null ┆ egg   ┆ 0.859469 ┆ C      ┆ 11      ┆ 5     │
│ 5    ┆ null  ┆ 0.020818 ┆ B      ┆ 11      ┆ 5     │
└──────┴───────┴──────────┴────────┴─────────┴───────┘

In [11]:
# Filter
out: pl.DataFrame = df_1.filter(pl.col("nrs") > 2)
print(out)

shape: (2, 6)
┌─────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ --- ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64 ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞═════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 3   ┆ spam  ┆ 0.575215 ┆ B      ┆ 11      ┆ 5     │
│ 5   ┆ null  ┆ 0.020818 ┆ B      ┆ 11      ┆ 5     │
└─────┴───────┴──────────┴────────┴─────────┴───────┘

### Group By

In [14]:
print(df_1)

shape: (5, 6)
┌──────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs  ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ ---  ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64  ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞══════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 1    ┆ foo   ┆ 0.938371 ┆ A      ┆ 11      ┆ 5     │
│ 2    ┆ ham   ┆ 0.11866  ┆ A      ┆ 11      ┆ 5     │
│ 3    ┆ spam  ┆ 0.575215 ┆ B      ┆ 11      ┆ 5     │
│ null ┆ egg   ┆ 0.859469 ┆ C      ┆ 11      ┆ 5     │
│ 5    ┆ null  ┆ 0.020818 ┆ B      ┆ 11      ┆ 5     │
└──────┴───────┴──────────┴────────┴─────────┴───────┘

In [12]:
out: pl.DataFrame = df_1.group_by("groups").agg(
    pl.sum("nrs"),  # sum nrs by groups
    pl.col("random").count().alias("count"),  # count group members
    # sum random where name != null
    pl.col("random").filter(pl.col("names").is_not_null()).sum().name.suffix("_sum"),
    pl.col("names").reverse().alias("reversed names"),
)
print(out)

shape: (3, 5)
┌────────┬─────┬───────┬────────────┬────────────────┐
│ groups ┆ nrs ┆ count ┆ random_sum ┆ reversed names │
│ ---    ┆ --- ┆ ---   ┆ ---        ┆ ---            │
│ str    ┆ i64 ┆ u32   ┆ f64        ┆ list[str]      │
╞════════╪═════╪═══════╪════════════╪════════════════╡
│ B      ┆ 8   ┆ 2     ┆ 0.575215   ┆ [null, "spam"] │
│ C      ┆ 0   ┆ 1     ┆ 0.859469   ┆ ["egg"]        │
│ A      ┆ 3   ┆ 2     ┆ 1.057032   ┆ ["ham", "foo"] │
└────────┴─────┴───────┴────────────┴────────────────┘

### Expressions

```text
- Polars has a powerful concept called expressions that is central to its very fast performance.
- Expressions are at the core of many data science operations:
  * taking a sample of rows from a column
  * multiplying values in a column
  * extracting a column of years from dates
  * convert a column of strings to lowercase

- However, expressions are also used within other operations:
  * taking the mean of a group in a group_by operation
  * calculating the size of groups in a group_by operation
  * taking the sum horizontally across columns
  * Polars performs these core data transformations very quickly by:

- Polars expressions are a mapping from a series to a series (or mathematically Fn(Series) -> Series). 
- As expressions have a Series as an input and a Series as an output then it is straightforward to do a sequence of expressions (similar to method chaining in Pandas).
```

In [15]:
# Select column "foo"
# Then sort the column (not in reversed order)
# Then take the first two values of the sorted output
pl.col("foo").sort().head(2)

In [18]:
df_1

nrs,names,random,groups,nrs_sum,count
i64,str,f64,str,i64,u32
1,"""foo""",0.938371,"""A""",11,5
2,"""ham""",0.11866,"""A""",11,5
3,"""spam""",0.575215,"""B""",11,5
null,"""egg""",0.859469,"""C""",11,5
5,null,0.020818,"""B""",11,5


In [19]:
df

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.938371,"""A"""
2,"""ham""",0.11866,"""A"""
3,"""spam""",0.575215,"""B"""
null,"""egg""",0.859469,"""C"""
5,null,0.020818,"""B"""


### [Lazy Vs Eager API](https://pola-rs.github.io/polars/user-guide/concepts/lazy-vs-eager/)

In [21]:
# Polars settings
pl.set_option("max_rows", 1000)
pl.set_option("max_columns", 1000)
pl.set_option("max_colwidth", 600)

AttributeError: module 'polars' has no attribute 'set_option'

In [30]:
fp: str = "../../data/AirlineTweets.csv"
df: pl.DataFrame = pl.read_csv(fp)
pl.Config.set_tbl_column_data_type_inline(True)
N: int = 8
with pl.Config(tbl_cols=N):  # display N columns
    print(df.head())

shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ tweet_id  ┆ airline_s ┆ airline_s ┆ negativer ┆ … ┆ tweet_coo ┆ tweet_cre ┆ tweet_loc ┆ user_tim │
│ (i64)     ┆ entiment  ┆ entiment_ ┆ eason     ┆   ┆ rd (str)  ┆ ated      ┆ ation     ┆ ezone    │
│           ┆ (str)     ┆ confidenc ┆ (str)     ┆   ┆           ┆ (str)     ┆ (str)     ┆ (str)    │
│           ┆           ┆ e (f64)   ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 570306133 ┆ neutral   ┆ 1.0       ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Eastern  │
│ 677760513 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:35:52  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301130 ┆ positive  ┆ 0.3486    ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 888122368 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:59  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301083 ┆ neutral   ┆ 0.6837    ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ Lets Play ┆ Central  │
│ 672813571 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:48  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301031 ┆ negative  ┆ 1.0       ┆ Bad       ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 407624196 ┆           ┆           ┆ Flight    ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:36  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570300817 ┆ negative  ┆ 1.0       ┆ Can't     ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 074462722 ┆           ┆           ┆ Tell      ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:14:45  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [31]:
df.describe()

describe,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
str,f64,str,f64,str,f64,str,str,str,str,f64,str,str,str,str,str
"""count""",14640.0,"""14640""",14640.0,"""14640""",14640.0,"""14640""","""14640""","""14640""","""14640""",14640.0,"""14640""","""14640""","""14640""","""14640""","""14640"""
"""null_count""",0.0,"""0""",0.0,"""5462""",4118.0,"""0""","""14600""","""0""","""14608""",0.0,"""0""","""13621""","""0""","""4733""","""4820"""
"""mean""",5.6922e17,null,0.900169,null,0.638298,null,null,null,null,0.08265,null,null,null,null,null
"""std""",7.7911e14,null,0.16283,null,0.33044,null,null,null,null,0.745778,null,null,null,null,null
"""min""",5.6759e17,"""negative""",0.335,"""Bad Flight""",0.0,"""American""","""negative""","""0504Traveller""","""Bad Flight""",0.0,"""""LOL you guys …","""[-33.87144962,…","""2015-02-16 23:…",""" || san anton…","""Abu Dhabi"""
"""25%""",5.6856e17,null,0.6923,null,0.3606,null,null,null,null,0.0,null,null,null,null,null
"""50%""",5.6948e17,null,1.0,null,0.6706,null,null,null,null,0.0,null,null,null,null,null
"""75%""",5.6989e17,null,1.0,null,1.0,null,null,null,null,0.0,null,null,null,null,null
"""max""",5.7031e17,"""positive""",1.0,"""longlines""",1.0,"""Virgin America…","""positive""","""zupshawrl""","""Lost Luggage D…",44.0,"""😳 LOLOLOLOLOL …","""[59.38247253, …","""2015-02-24 11:…","""명동서식 37.56638,…","""West Central A…"


In [32]:
VALUE: float = 0.567
df_small: pl.DataFrame = df.filter(pl.col("airline_sentiment_confidence") > VALUE)
df_agg: pl.DataFrame = df_small.group_by("negativereason").agg(
    pl.col("retweet_count").mean()
)
print(df_agg)

shape: (11, 2)
┌─────────────────────────────┬─────────────────────┐
│ negativereason (str)        ┆ retweet_count (f64) │
╞═════════════════════════════╪═════════════════════╡
│ longlines                   ┆ 0.050562            │
│ null                        ┆ 0.065797            │
│ Flight Attendant Complaints ┆ 0.090336            │
│ Flight Booking Problems     ┆ 0.080614            │
│ …                           ┆ …                   │
│ Customer Service Issue      ┆ 0.076128            │
│ Cancelled Flight            ┆ 0.07346             │
│ Can't Tell                  ┆ 0.187983            │
│ Late Flight                 ┆ 0.093656            │
└─────────────────────────────┴─────────────────────┘